# AML Challenge 1 Group 2

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
from tqdm.notebook import tqdm 

import matplotlib
from matplotlib import rc


rc("font", **{"family": "sans-serif", "sans-serif": "DejaVu Sans"})
rc("figure", **{"dpi": 200})
rc(
    "axes",
    **{"spines.right": False, "spines.top": False, "xmargin": 0.0, "ymargin": 0.05}
)

In [ ]:
!nvidia-smi -L

In [ ]:
#!pip install wandb
#import wandb
#wandb.init()

In [ ]:
df = pd.read_csv('../input/eurecom-aml-2022-challenge-1/public/train.csv', low_memory=True)

In [ ]:
df = df.sample(frac=1)

In [ ]:
df_test = pd.read_csv('../input/eurecom-aml-2022-challenge-1/public/test_feat.csv', low_memory=True)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 3000) #pd.options.display.max_rows = 4000


In [ ]:
df_concat = pd.concat([df, df_test], axis=0)


## Data pre-processing 
One of the feature engineerings we did to prepare the data was to seperate the different latitudes into "bulks" describing how far from equator the data point is, called eq_dist. If the datapoint has a latitude more than -5 or less than 5 it is on equator, and gets the eq_dist equal to 0. If it is not in that range, but still in the [-10,10] range it will get a value equal to 1. This goes for all the ranges down to -90 and up to 90. We did not end up using it tho, thats why its commented out. 

We also reformatted the timestamp, so we had month, day and hour instead of the unix timestamp, and removed nan-values. The converstion of timestamp makes sure that the model can extract useful information like what hour of the day it is and which monthn youre in.

In [ ]:
#import math
#def calc_dist(val):
#    ps = [range(0, 5), range(5,10), range(10, 15), range(15, 20), range(20, 25),range(25,30),range(30, 35),range(35,40), range(40,45), range(45,50), range(50,55), range(55,60), range(60,65), range(65,70), range(70, 75),range(75,80),range(80,85), range(85, 90)]
#    for i in range(len(ps)): 
#        if math.ceil(abs(val)) in ps[i]:
#            return i 
#    return 18
#df_concat["eq_dist"] = df_concat.apply(lambda x: calc_dist(val=x["fact_latitude"]), axis=1)

In [ ]:
# Here we do the feature engineering on the timestamp value
df_concat['fact_time'] = pd.to_datetime(df_concat['fact_time'],unit='s')
df_concat['month'] = pd.DatetimeIndex(df_concat['fact_time']).month
df_concat['day'] = pd.DatetimeIndex(df_concat['fact_time']).day
df_concat['hour'] = pd.DatetimeIndex(df_concat['fact_time']).hour
print(df_concat.isna().sum())

# Dropping CMC and WRF non-values (represented by the two columns given)
df_concat_nn = df_concat[df_concat["cmc_precipitations"].notnull()]
df_concat_nn_nn = df_concat_nn[df_concat_nn["wrf_t2_next"].notnull()]
print(df_concat_nn_nn.isna().sum())

Further we want to look at the correlated columns, to see if there are columns contributing "the same information" (highly correlated). This way we could drop the related

In [ ]:
#Preparing the training set which will be used for finding correlation 

df['fact_time'] = pd.to_datetime(df['fact_time'],unit='s')
df['month'] = pd.DatetimeIndex(df['fact_time']).month
df['day'] = pd.DatetimeIndex(df['fact_time']).day
df['hour'] = pd.DatetimeIndex(df['fact_time']).hour

In [ ]:
def calc_most_correlated(df, corr_value): 
    corr_df = df.corr().abs()
    filtered_corr_df = corr_df[(corr_df>=corr_value) & (corr_df != 1.000)]
    correlated = {}
    for index, row in filtered_corr_df.iterrows():
        c = [filtered_corr_df.columns[i] for i in range(len(filtered_corr_df.columns)) if not np.isnan(row[i])]
        if len(c)>0: 
            correlated[index]=c
    corr_sort = sorted(correlated, key=lambda k: len(correlated[k]), reverse=True)
    return corr_sort, correlated


corr_sort, correlated = calc_most_correlated(df, 0.95)


In [ ]:
def find_columns_to_drop(corr_sort, correlated):
    dropped = set()
    for value in corr_sort: 
        if value not in dropped: 
            dropped = dropped.union({key for key in correlated[value]})
    return dropped

#corr_sort = calc_most_correlated(df, 0.95)
#dropped = find_columns_to_drop(corr_sort, correlated)

The following commented out line of code was ment to be used to drop the not so important (correlating) columns, but we ended up using all columns.

In [ ]:
#df_concat_nn_nn_dropped = df_concat_nn_nn.drop(dropped)

The following code is used to create clusters as described in the powerpoint.


In [ ]:
#in order for the KMEANS to run in reasonable time we had to just select the most important columns
col_selection = [
    'index', 'fact_latitude', 'fact_longitude',
    'topography_bathymetry', 'sun_elevation', 'cmc_precipitations', 
    'gfs_a_vorticity', 'gfs_cloudness', 'gfs_clouds_sea', 'gfs_humidity',
    'fact_temperature', 'month', 'day', 'hour', 'cmc_0_0_6_2', 'cmc_0_0_7_2', 'wrf_rh2'
]
df_selected = df_concat_nn_nn[col_selection]

In [ ]:
def lat_long_clustering(df):
    from sklearn.cluster import KMeans
    k_means = KMeans(n_clusters = 35, max_iter = 20, init='k-means++', random_state=42, verbose=1)
    df['fact_latitude'] = df['fact_latitude']*3 #multiply by three to improve importance of latitude
    lat_long_pairs = df[['fact_latitude','fact_longitude']]
    #lat_long_pairs['fact_longitude'] = lat_long_pairs['fact_longitude']*3
    target = (df[df['fact_temperature'].notnull()].fact_temperature)
    
    k_means.fit(df[df['fact_temperature'].notnull()][['fact_latitude','fact_longitude']],sample_weight = target)
    df['lat_long_cluster'] = k_means.predict(lat_long_pairs)

    return df

df_selected_clustered = lat_long_clustering(df_selected)


In [ ]:
df_concat_nn_nn["lat_long_cluster"] = df_selected_clustered["lat_long_cluster"]

# PCA
Commented out becaus it was not used in the end

In [ ]:
#X = StandardScaler().fit_transform(X)
#y = StandardScaler().fit_transform(y.reshape(-1, 1)).squeeze()

#%%time
#pca1 = PCA(n_components=1)
#pca90 = PCA(n_components=0.9)
#pca95 = PCA(n_components=0.95)
#pca99 = PCA(n_components=0.99)
#pca1.fit(X)
#pca90.fit(X)
##pca95.fit(X)
#pca99.fit(X)

#print(f"PCA 90% component variance: {len(pca90.explained_variance_ratio_)}")
#print(f"PCA 95% component variance: {len(pca95.explained_variance_ratio_)}")
#print(f"PCA 99% component variance: {len(pca99.explained_variance_ratio_)}")

#X_1 = pca1.transform(X)
#X_90 = pca90.transform(X)
#X_95 = pca95.transform(X)
#X_99 = pca99.transform(X)

In [ ]:
#X_1.shape

#plt.title("Most variant PCA feature")
#plt.scatter(X_1, y, s=2, c="#50BEA8")
#plt.ylabel("Temperature")
#plt.show()

## Train test set

In [ ]:
X = df_concat_nn_nn.drop("fact_time", axis=1)

In [ ]:
test = X[X['fact_temperature'].isnull()]
train = X[X['fact_temperature'].notnull()]
train

In [ ]:
import sys
!{sys.executable} -m pip install xgboost
import xgboost as xgb

In [ ]:
X = train.drop("fact_temperature", axis=1)
y = train["fact_temperature"]
X

# Test of Ensamle-model
We did not end up using this, hence commented out.

In [ ]:
SEED=42
# model1 = RandomForestRegressor(
#     n_estimators=1500,
#     n_jobs=-1,
#     random_state=SEED,
#     verbose=1,
# )
# model2 = GradientBoostingRegressor(
#     n_estimators=600,
#     learning_rate=0.06,
#     min_samples_leaf=4, 
#     max_depth=9, 
#     random_state=SEED,
#     verbose=1,
# )
# model3 = lgb.LGBMRegressor(
#     n_estimators=6000,
#     learning_rate=0.08,
#     num_leaves=10,
#     random_state=SEED, 
#     seed=SEED,
#     n_jobs=-1,
# )
# model4 = xgb.XGBRegressor(
#     n_estimators=11000,
#     learning_rate=0.01,
#     n_jobs=-1, 
#     subsample=0.8,
#     random_state=SEED,
#     max_depth = 8,
#     gamma=0.0,
#     seed=SEED,
#     verbosity=1,
#     tree_method='gpu_hist'
# )
# model5 = CatBoostRegressor(
#     n_estimators=5000,
#     learning_rate=0.06322764426255192,
#     thread_count=-1,
#     num_leaves=24,
#     min_child_samples=16,
#     depth=6,
#     task_type="GPU",
#     random_seed=SEED,
#     silent=True,
#     grow_policy='Lossguide',
# )

In [ ]:
#X_test = test.drop("fact_temperature", axis=1)

In [ ]:
#from sklearn.model_selection import KFold

In [ ]:
# ntrain = X.shape[0]
# ntest = X_test.shape[0]
# SEED = 42 # for reproducibility
# NFOLDS = 3 # set number of folds for out-of-fold prediction
# kf = KFold(
#     n_splits=NFOLDS,
#     shuffle=True,
#     random_state=SEED
# ) # K-Folds cross-validator

# def get_oof(clf, x_train, y_train, x_test):
#     """
#     Popular function on Kaggle.
    
#     Trains a classifier on 4/5 of the training data and
#     predicts the rest (1/5). This procedure is repeated for all 5 folds,
#     thus we have predictions for all training set. This prediction is one
#     column of meta-data, later on used as a feature column by a meta-algorithm.
#     We predict the test part and average predictions across all 5 models.
    
#     Keyword arguments:
#     clf -- classifier
#     x_train -- 4/5 of training data
#     y_train -- corresponding labels
#     x_test -- all test data
    
#     """
#     oof_train = np.zeros((ntrain,))
#     oof_test = np.zeros((ntest,))
#     oof_test_skf = np.empty((NFOLDS, ntest))

#     for i, (train_index, test_index) in enumerate(kf.split(x_train)):
#         x_tr = x_train[train_index]
#         y_tr = y_train[train_index]
#         x_te = x_train[test_index]

#         clf.fit(x_tr, y_tr)

#         oof_train[test_index] = clf.predict(x_te)
#         oof_test_skf[i, :] = clf.predict(x_test)

#     oof_test[:] = oof_test_skf.mean(axis=0)
#     return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
# X_train = X.values
# X_test = X_test.values
# y_train = y.ravel()

In [ ]:
# lgb_oof_train, lgb_oof_test = get_oof(model3, X_train, y_train, X_test)

In [ ]:
# xgb_oof_train, xgb_oof_test = get_oof(model4, X_train, y_train, X_test) 

In [ ]:
# cat_oof_train, cat_oof_test = get_oof(model5, X_train, y_train, X_test)

In [ ]:
# x_train = np.concatenate((
#     lgb_oof_train,
#     xgb_oof_train,
#     cat_oof_train
# ), axis=1)

# x_test = np.concatenate((
#     lgb_oof_test,
#     xgb_oof_test,
#     cat_oof_test
# ), axis=1)

In [ ]:
# from sklearn.linear_model import RidgeCV

In [ ]:
# META_MODEL = RidgeCV(cv=5)
# META_MODEL.fit(x_train, y_train)
# final_predictions = META_MODEL.predict(x_test)

# # final_predictions = np.average(
# #     [
# #      rf_oof_test,
# #      gb_oof_test,
# #      lgb_oof_test,
# #      xgb_oof_test,
# #      cat_oof_test
# #     ],
# #     weights = 1 / acc['RMSLE']**9,
# #     axis=0
# # )
# # final_predictions = np.power(2, final_predictions)
# final_predictions

In [ ]:
# submission_df = pd.DataFrame(data={'index': test['index'].values,
#                                    'fact_temperature': final_predictions.squeeze()})

# # Save the predictions into a csv file
# # Notice that this file should be saved under the directory `/kaggle/working` 
# # so that you can download it later
# submission_df.to_csv("submission_ensamble3.csv", index=False)

In [ ]:
# kaggle competitions submit -c eurecom-aml-2022-challenge-1 -f submission_ensamble3.csv -m "XGB CAT and LGBM with RidgeCV"

In [ ]:
# ! head -6 "submission_ensamble2.csv"

# Submission

In [ ]:
# #xgb_model = xgb.XGBRegressor(n_jobs=-1, verbosity=2, max_depth=9, n_estimators=11000, learning_rate=0.01, tree_method="gpu_hist", seed=42)
xgb_model = xgb.XGBRegressor(n_jobs=-1, verbosity=2, n_estmators=6500, max_depth=10, learning_rate=0.14, min_child_wheight=30, tree_method="gpu_hist", seed=42)
# xgb_model = xgb.XGBRegressor(n_jobs=-1, verbosity=2, n_estmators=7000, max_depth=10, learning_rate=0.1, min_child_wheight=2, tree_method="gpu_hist", seed=42)

xgb_model.fit(X,y)

Plots used to find feature importance of XGBoost

In [ ]:
# axes = plt. gca()
# axes. xaxis. label. set_size(100)
# axes. yaxis. label. set_size(100)
# plt.barh(X.columns[:20], xgb_model.feature_importances_[:20])


In [ ]:
from xgboost import plot_importance
plt.rcParams["figure.figsize"] = (200,200)
plot_importance(xgb_model)

In [ ]:
prediction = xgb_model.predict(test.drop("fact_temperature", axis=1))

In [ ]:
submission_df = pd.DataFrame(data={'index': df_test['index'].values,
                                   'fact_temperature': prediction.squeeze()})

# # Save the predictions into a csv file
# # Notice that this file should be saved under the directory `/kaggle/working` 
# # so that you can download it later
ubmission_df.to_csv("submission.csv", index=False)

In [ ]:
! head -6 "submission.csv"

In [ ]:
tr_coordinates = train[['fact_latitude','fact_longitude','lat_long_cluster']].drop_duplicates()
te_coordinates = test[['fact_latitude','fact_longitude','lat_long_cluster']].drop_duplicates()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=[10, 3], dpi=200, sharex=True, sharey=True)
ax0.scatter(tr_coordinates['fact_longitude'], tr_coordinates['fact_latitude'], 
            s=1, label='Train data',c=tr_coordinates.lat_long_cluster, cmap='viridis' ) #c=tr_coordinates.fact_temperature
ax0.legend()
ax1.scatter(te_coordinates['fact_longitude'], te_coordinates['fact_latitude'], 
            s=1, label='Test data',c=te_coordinates.lat_long_cluster, cmap='viridis' )
ax1.legend()
fig.suptitle('Distribution of train and test points in the dataset', y=1.01, fontsize=14)
plt.show()

# KFold validation and Optuna Tuning

In [ ]:
from copy import deepcopy
from sklearn.model_selection import KFold 
def compute_rmse(y, ypred, ystd=1.):
    return np.mean((y - ypred)**2)**0.5 * ystd

Following is a simple KFold used to check model performance locally.

In [ ]:
#
#Implementing cross validation
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=42)
#new parametersxgb_model = xgb.XGBRegressor(n_jobs=-1, verbosity=2, max_depth=10, n_estimators=6500, learning_rate=0.14, min_child_weigth=30,tree_method="gpu_hist", seed=42)
model = xgb.XGBRegressor(n_jobs=-1, verbosity=2,  max_depth=10, n_estimators=6500, learning_rate=0.14, min_child_weigth=30, tree_method="gpu_hist", seed=42)
target = deepcopy(y.to_numpy())
X = deepcopy(X)
acc_score = []

for train_index , test_index in kf.split(X):
    X_train_k , X_test_k = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train_k , y_test_k = target[train_index] , target[test_index]
    
    model.fit(X_train_k,y_train_k)
  
    pred_values = model.predict(X_test_k)
    acc = compute_rmse(y=y_test_k, ypred=pred_values)
    acc_score.append(acc)
    
avg_acc_score = sum(acc_score)/k
print(avg_acc_score)
#1.8631868495864443

In [ ]:
pip install optuna

Following is the Optuna-code used to train XGBoost, in combination with KFold

In [ ]:
import optuna
import lightgbm as lgb
from copy import deepcopy
from sklearn.model_selection import KFold 
def objective(trial,data=X,target=y):
 
    #Loading the dataset

    y = deepcopy(target.to_numpy())
    X = deepcopy(data)
    
    #Implementing cross validation
    k = 4
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    param = {
        'n_estimators': trial.suggest_categorical('n_estmators', [5000,6500,7000,9000,10500]),
        'max_depth': trial.suggest_categorical('max_depth', [8,10,12, 14]),
        'learning_rate':trial.suggest_categorical('learning_rate', [0.06,0.1, 0.14, 0.3]),
        #'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', [1,2,4,6,9,12]),
        #'num_leaves': trial.suggest_categorical('num_leaves', [4,17,21,24,27,50]),
        #'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart']),
        #'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),'reg:squaredlogerror',
        #'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
        'min_child_wheight': trial.suggest_categorical('min_child_wheight', [0,0.5,2,6,15,30]),
        'n_jobs': -1,
        'seed': 42,
        'tree_method': "gpu_hist",
    }

    model = xgb.XGBRegressor(**param)
    
    acc_score = []

    for train_index , test_index in kf.split(X):
        X_train_k , X_test_k = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train_k , y_test_k = y[train_index] , y[test_index]
        
        model.fit(X_train_k,y_train_k)
      
        pred_values = model.predict(X_test_k)
        acc = compute_rmse(y=y_test_k, ypred=pred_values)
        acc_score.append(acc)
        
    avg_acc_score = sum(acc_score)/k
      
    return avg_acc_score

optuna.logging.enable_default_handler()
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

study.trials_dataframe()
optuna.visualization.plot_param_importances(study)
optuna.visualization.plot_optimization_history(study)